In [139]:
# Import packages
import pandas as pd
import numpy 
import os
import pdfplumber
import re

In [140]:
# Define a function to read in text from pdfs

def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text()
    return text

In [141]:
# Define a function to extract the relevant data

def extract_data(text):
    number_pattern = r"Invoice Number:\s*(\d*)"
    number = re.findall(number_pattern,text)[0]
    
    date_pattern = r"Invoice Date:\s*(\b\d{2}[/\-.]\d{2}[/\-.]\d{4}\b)"
    date = re.findall(date_pattern,text)[0]
    
    total_pattern = r"Total \w*\s*(£\d{1,4}\.\d{2})"
    total = re.findall(total_pattern,text)[0]
    
    return number, date, total

In [142]:
# Change the wd to read in the data files
os.chdir('./Data')

In [143]:
# Save the file names to a variable
file_names = os.listdir()

In [144]:
# Import all of the files and add the date

# Create a df to save data
solar = pd.DataFrame(columns=['file', 'invoice_number', 'invoice_date', 
                              'amount_total'])

# Iterate through the files
for file in file_names:
    # Open the files using the pdf function
    text = extract_text_from_pdf(file)
    # Extract the data using the text function
    number, date, total = extract_data(text)
    
    new_row = [file, number, date, total]
    solar.loc[len(solar)] = new_row
    

# Set the date field to a date field
solar['invoice_date'] = pd.to_datetime(solar['invoice_date'], format='%d.%m.%Y')
# Sort by invoice data
solar = solar.sort_values(by = 'invoice_date')

In [145]:
# Display the top of the df
solar.head()

,file,invoice_number,invoice_date,amount_total
36,FIT_803B2C15112010_181110_113933.pdf,1900005075,2010-11-16,£258.94
29,FIT_803B2C24022011_020311_110050.pdf,1900001602,2011-02-24,£135.68
6,FIT_803B2C10052011_130511_081522.pdf,1900004998,2011-05-12,£54.78
28,FIT_803B2C11082011_120811_094035.pdf,1900009345,2011-08-11,£303.03
45,FIT_803B2C21112011_211111_145211.pdf,1900015792,2011-11-21,£338.62
